In [3]:
!pip install -U pymgrid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 22.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for pymgrid: filename=pymgrid-1.2.2-py3-none-any.whl size=3492743 sha256=417b92d115cddc3dea8f8cd137b7cc65fa35d4d152d4a064aa29a3ac99c821d1
  Stored in directory: /root/.cache/pip/wheels/0b/d9/2d/02835d5a5c120b07bfa8d455e9efa2a8bb4dd56acb7255e69d
Successfully built pymgrid


In [4]:
import numpy as np
import pandas as pd

np.random.seed(0)

from pymgrid import Microgrid
from pymgrid.modules import (
    BatteryModule,
    LoadModule,
    RenewableModule,
    GensetModule,
    GridModule)
from pymgrid.algos import ModelPredictiveControl

In [5]:
import pandas as pd

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [34]:
import plotly.express as px

*We* can then define some components of our microgrid. We will define two battery with a slower charging rate and lower efficiency but a capacity of 1000 kWh.

In [6]:
large_battery = BatteryModule(min_capacity=10,
                              max_capacity=1000,
                              max_charge=10,
                              max_discharge=10,
                              efficiency=0.7,
                              init_soc=0.2)

In [7]:
load_df = pd.read_csv("/content/Shan_21.4148_98.1206_142.csv")
load_df_hourly =  load_df.iloc[::60]

In [8]:
load_pv_sim = pd.read_csv("/content/shan_myanmar_pv_21.5122_98.0098_uncorrected.csv")
load_pv_sim['electricity']
pv_sim_array_inp = np.array(load_pv_sim['electricity'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [9]:
pv_sim_array = np.hstack((pv_sim_array_inp,pv_sim_array_inp,pv_sim_array_inp,pv_sim_array_inp))
np.shape(pv_sim_array)
pv_ts = pv_sim_array

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [10]:
load_df_hourly

,timestamp,date,time,kW
0,2022-01-01 00:00:00,2022-01-01,00:00,4.000000
60,2022-01-01 01:00:00,2022-01-01,01:00,4.000000
120,2022-01-01 02:00:00,2022-01-01,02:00,4.000000
180,2022-01-01 03:00:00,2022-01-01,03:00,4.000000
240,2022-01-01 04:00:00,2022-01-01,04:00,4.000000
...,...,...,...,...
525300,2022-12-31 19:00:00,2022-12-31,19:00,18.194728
525360,2022-12-31 20:00:00,2022-12-31,20:00,20.098362
525420,2022-12-31 21:00:00,2022-12-31,21:00,14.321299
525480,2022-12-31 22:00:00,2022-12-31,22:00,7.133272


In [11]:
load_df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,timestamp,date,time,kW
0,2022-01-01 00:00:00,2022-01-01,00:00,4.0
1,2022-01-01 00:01:00,2022-01-01,00:01,4.0
2,2022-01-01 00:02:00,2022-01-01,00:02,4.0
3,2022-01-01 00:03:00,2022-01-01,00:03,4.0
4,2022-01-01 00:04:00,2022-01-01,00:04,4.0


In [12]:
load_ts_myanmar = np.array(load_df_hourly['kW'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [13]:
load = LoadModule(time_series=load_ts_myanmar)
pv = RenewableModule(time_series=pv_ts)

We will define an external electrical grid to fill any energy gaps.

The grid time series must contain three or four columns. The first three denote import price, export price, and carbon dioxide production per kWh.

In [14]:
test_size = np.shape(load_ts_myanmar)[0]
grid_ts = [0.2, 0.1, 0.5] * np.ones((test_size, 3))
grid = GridModule(max_import=100,
                  max_export=100,
                  time_series=grid_ts)

*We* can then pass these to the ``Microgrid`` constructor to define a microgrid. Here, we give our renewable module the name "pv".

In [15]:
modules = [
    large_battery,
    ('pv', pv),
    load,
    grid
    ]

microgrid = Microgrid(modules)

In [16]:
mpc = ModelPredictiveControl(microgrid)

We can then simply define a control for these modules. Before doing so, we will reset our microgrid and then check its current state.

In [17]:
microgrid.reset()
microgrid.state_series.to_frame()

0
load    0 load_current           -4.000
pv      0 renewable_current     654.462
battery 0 soc                     0.200
          current_charge        200.000
grid    0 import_price_current    0.200
          export_price_current    0.100
          co2_per_kwh_current     0.500
          grid_status_current     1.000

In [18]:
mpc_result = mpc.run()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



### Analyzing Results

After passing a control to the microgrid, we can view the results by viewing the microgrid's -- or any of the modules -- logs.

The microgrid's log has one row for each action taken. There are values for both the actions -- e.g. the amount of load met -- as well as the state: for example, the current load.

In [19]:
output= mpc_result

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [21]:
load_out_df = output['load'][0]
load_out_df.to_csv('load_out_df', index=False)
del load_out_df['reward']


In [22]:
pv_out_df = output['pv'][0]
del pv_out_df['reward']
pv_out_df = pv_out_df.rename(columns={"curtailment": "pv_curtailment", "renewable_used": "pv_renewable_used","renewable_current": "pv_renewable_current"})
pv_out_df

field,pv_curtailment,pv_renewable_used,pv_renewable_current
0,650.462000,4.000000,654.462
1,1409.973000,4.000000,1413.973
2,1840.891000,4.000000,1844.891
3,2262.514000,4.000000,2266.514
4,2287.533158,6.088842,2293.622
...,...,...,...
8754,0.000000,-0.000000,0.000
8755,0.000000,-0.000000,0.000
8756,0.000000,-0.000000,0.000
8757,0.000000,-0.000000,0.000


In [23]:
balancing_out_df = output['balancing'][0]
del balancing_out_df['reward']
balancing_out_df.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



field,loss_load,overgeneration
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


In [24]:
battery_0_out_df = output['battery'][0]
del battery_0_out_df['reward']
del battery_0_out_df['soc']
#del battery_0_out_df['field']
battery_0_out_df = battery_0_out_df.rename(columns={"discharge_amount": "battery_0_discharge_amount", "charge_amount": "battery_0_charge_amount","renewable_current": "battery_0_renewable_current"})
battery_0_out_df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



field,battery_0_discharge_amount,battery_0_charge_amount,current_charge
0,0.0,0.0,200.0
1,0.0,0.0,200.0
2,0.0,0.0,200.0
3,0.0,0.0,200.0
4,0.0,0.0,200.0


In [25]:
balance_out_df = output['balance'][0]
del balance_out_df['reward']
balance_out_df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



field,overall_provided_to_microgrid,overall_absorbed_from_microgrid,controllable_provided_to_microgrid,controllable_absorbed_from_microgrid,fixed_provided_to_microgrid,fixed_absorbed_from_microgrid
0,4.000000,4.000000,0.0,0.0,0.0,4.000000
1,4.000000,4.000000,0.0,0.0,0.0,4.000000
2,4.000000,4.000000,0.0,0.0,0.0,4.000000
3,4.000000,4.000000,0.0,0.0,0.0,4.000000
4,6.088842,6.088842,0.0,0.0,0.0,6.088842


In [26]:
external_grid_out_df = output['grid'][0]
del external_grid_out_df['reward']
external_grid_out_df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



field,co2_production,grid_import,grid_export,import_price_current,export_price_current,co2_per_kwh_current,grid_status_current
0,0.0,0.0,0.0,0.2,0.1,0.5,1.0
1,0.0,0.0,0.0,0.2,0.1,0.5,1.0
2,0.0,0.0,0.0,0.2,0.1,0.5,1.0
3,0.0,0.0,0.0,0.2,0.1,0.5,1.0
4,0.0,0.0,0.0,0.2,0.1,0.5,1.0


In [27]:
del external_grid_out_df['grid_export']
del external_grid_out_df['import_price_current']
del external_grid_out_df['export_price_current']
del external_grid_out_df['co2_per_kwh_current']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [28]:
del external_grid_out_df['grid_status_current']

In [29]:
external_grid_out_df

field,co2_production,grid_import
0,0.000000,0.000000
1,0.000000,0.000000
2,0.000000,0.000000
3,0.000000,0.000000
4,0.000000,0.000000
...,...,...
8754,9.097364,18.194728
8755,10.049181,20.098362
8756,7.160650,14.321299
8757,3.566636,7.133272


In [30]:
#result_df = pd.concat([load_out_df,wind_out_df], axis=1)#battery_0_out_df,battery_1_out_df,balancing_out_df,balance_out_df,external_grid_out_df])
result_df = pd.concat([load_out_df,pv_out_df,battery_0_out_df,balancing_out_df,balance_out_df,external_grid_out_df], axis=1)
result_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



field,load_met,load_current,pv_curtailment,pv_renewable_used,pv_renewable_current,battery_0_discharge_amount,battery_0_charge_amount,current_charge,loss_load,overgeneration,overall_provided_to_microgrid,overall_absorbed_from_microgrid,controllable_provided_to_microgrid,controllable_absorbed_from_microgrid,fixed_provided_to_microgrid,fixed_absorbed_from_microgrid,co2_production,grid_import
0,4.000000,-4.000000,650.462000,4.000000,654.462,0.0,0.0,200.0,0.0,0.0,4.000000,4.000000,0.000000,0.0,0.0,4.000000,0.000000,0.000000
1,4.000000,-4.000000,1409.973000,4.000000,1413.973,0.0,0.0,200.0,0.0,0.0,4.000000,4.000000,0.000000,0.0,0.0,4.000000,0.000000,0.000000
2,4.000000,-4.000000,1840.891000,4.000000,1844.891,0.0,0.0,200.0,0.0,0.0,4.000000,4.000000,0.000000,0.0,0.0,4.000000,0.000000,0.000000
3,4.000000,-4.000000,2262.514000,4.000000,2266.514,0.0,0.0,200.0,0.0,0.0,4.000000,4.000000,0.000000,0.0,0.0,4.000000,0.000000,0.000000
4,6.088842,-6.088842,2287.533158,6.088842,2293.622,0.0,0.0,200.0,0.0,0.0,6.088842,6.088842,0.000000,0.0,0.0,6.088842,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8754,18.194728,-18.194728,0.000000,-0.000000,0.000,0.0,0.0,10.0,0.0,0.0,18.194728,18.194728,18.194728,0.0,0.0,18.194728,9.097364,18.194728
8755,20.098362,-20.098362,0.000000,-0.000000,0.000,0.0,0.0,10.0,0.0,0.0,20.098362,20.098362,20.098362,0.0,0.0,20.098362,10.049181,20.098362
8756,14.321299,-14.321299,0.000000,-0.000000,0.000,0.0,0.0,10.0,0.0,0.0,14.321299,14.321299,14.321299,0.0,0.0,14.321299,7.160650,14.321299
8757,7.133272,-7.133272,0.000000,-0.000000,0.000,0.0,0.0,10.0,0.0,0.0,7.133272,7.133272,7.133272,0.0,0.0,7.133272,3.566636,7.133272


In [31]:
result_df.to_csv('Grid_simulation_results_Shan.csv', index=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [40]:

# Plot the output
cols = [
    ("load", 0, "load_current"),
    ("load", 0, "load_met"),
    ("pv", 0, "renewable_used"),
    ("pv", 0, "curtailment"),
    ("pv", 0, "renewable_current"),
    ("battery", 0, "current_charge"),
    ("battery", 0, "discharge_amount"),
    ("battery", 0, "charge_amount"),
    #("battery", 1, "current_charge"),
    #("battery", 1, "discharge_amount"),
    #("battery", 1, "charge_amount"),
    ("grid", 0, "grid_import"),
    ("grid", 0, "grid_export"),
    ("grid", 0, "grid_status_current"),
    ("balancing", 0, "overgeneration"),
    ("balancing", 0, "loss_load"),
]

In [36]:
output.columns

MultiIndex([(     'load', 0,                               'reward'),
            (     'load', 0,                             'load_met'),
            (     'load', 0,                         'load_current'),
            (       'pv', 0,                               'reward'),
            (       'pv', 0,                          'curtailment'),
            (       'pv', 0,                       'renewable_used'),
            (       'pv', 0,                    'renewable_current'),
            ('balancing', 0,                               'reward'),
            ('balancing', 0,                            'loss_load'),
            ('balancing', 0,                       'overgeneration'),
            (  'battery', 0,                               'reward'),
            (  'battery', 0,                     'discharge_amount'),
            (  'battery', 0,                        'charge_amount'),
            (  'battery', 0,                                  'soc'),
            (  'batt

In [41]:
flat = output[cols]
flat.columns = ["_".join(map(str, col)).strip() for col in flat.columns.values]
flat["load_0_load_current"] = -flat["load_0_load_current"]
cols = flat.columns
fig = px.line(flat, y=cols, x=list(range(len(flat))))
fig.show()


<ipython-input-41-fee5fc7eadb1>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

